In [44]:
####################################################################################################
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carrega o dataset Iris
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names) # organizando em um df
df['target'] = iris.target # rotulos

X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)
modelo = LogisticRegression(max_iter=200)
modelo.fit(X_train, y_train) # treinando o modelo


w = modelo.coef_[0] # pesos do modelo treinado
Vs = X_test.iloc[0].to_dict() # utilizxando a primeira instância dos dados de teste 

delta = []

# Calcula o valor delta para cada feature
# peso negativo, valor menor = maior possibilidade da instância peetencer a classe
# delta c\ peso positivo é (valor feature na instancia - valor min da feature no dataset) * o peso da feature 
# delta c\ peso negativo é (vvalor max da feature no dataset - valor feature na instancia - ) * o peso da feature
for i, feature in enumerate(df.columns[:-1]):  
    if w[i] < 0:
        delta.append((Vs[feature] - df[feature].max()) * w[i])
    else:
        delta.append((Vs[feature] - df[feature].min()) * w[i])

# Calcula o limiar (-Gamma_w)

R = sum(delta) # "(Vs predição da regressão loghistica)  # Usando a equação 13 do artigo e garantindo que R seja positivo
# R = O valor absoluto da soma de todos os deltas

def one_explanation(Vs, delta, R):
    Xpl = []  # Inicializa a lista de PI-explicação
    # Ordena o delta junto com seus índices, em ordem decrescente de valor absoluto
    print(delta)
    delta_sorted = sorted(enumerate(delta), key=lambda x: abs(x[1]), reverse=True) # enumerando os valores de delta em tuplas e ordenanas
    print(delta_sorted)                                                                               # func lambda recebe a tupla produzida com o valor absoluto em ordem decrescente, do maior para o menor.
    R_atual = R  # Inicializa o limiar atual
    Idx = 0  # Inicializa o índice para iterar
    
    # Limite para considerar uma feature como importante, com base no limar R
    # threshold_delta = 0 * R # Valor de delta mínimo para ser considerado relevante
    
    while R_atual >= 0 and Idx < len(delta_sorted):
        sorted_idx, delta_value = delta_sorted[Idx]  # Desempacota o índice(soted_idx) e o valor(delta_value) do delta de acordo com o indce(idx)
        feature = X_test.columns[sorted_idx]  # Obtém o nome da feature correspondente
        feature_value = Vs[feature]  # Obtém o valor da feature na instância Vs
        
        # Adiciona à explicação apenas se o delta for maior que o threshold que está sendo calculado com uma % do R. não sei se esta certo isso
       # if abs(delta_value) > threshold_delta:  # Verifica se o delta tem impacto relevante 
        Xpl.append(f"{feature} - {feature_value}")  # Adiciona feature e valor à explicação
        
        R_atual -= delta_value  # Atualiza o limiar atual para manter ou parar o loop
        Idx += 1  # Incrementa o índice também para o loop
    
    return Xpl  # Retorna a PI-explicação

# Computa a PI-explicação
Xpl = one_explanation(Vs, delta, R)

#print(f"Pesos: {w}")
#print(f"Delta: {delta}")
#print(f"Limiar R: {R:.2f}")
#print(f"Vs: {Vs}\n")
print(f"PI-Explicação: ")
for item in Xpl:
    print(f"- {item}")

[0.7088652578943647, 0.7697645172444486, 5.2254240521431345, 1.2983454980344349]
[(2, 5.2254240521431345), (3, 1.2983454980344349), (1, 0.7697645172444486), (0, 0.7088652578943647)]
PI-Explicação: 
- petal length (cm) - 4.7
- petal width (cm) - 1.2
- sepal width (cm) - 2.8
- sepal length (cm) - 6.1


In [5]:
#ANALISANDO COMPLETO UMA ÚNICA INSTÂNCIA PARA O DATASET IRIS
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carrega o dataset Iris
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)  # organizando em um DataFrame
df['target'] = iris.target  # rótulos

# Divide os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

# Treina o modelo
modelo = LogisticRegression(max_iter=200)
modelo.fit(X_train, y_train)  # treinando o modelo

# Usa a primeira instância dos dados de teste para a explicação
Vs = X_test.iloc[0].to_dict()  # utilizando a primeira instância dos dados de teste 
instancia_test = X_test.iloc[[0]]  # Mantem como DataFrame para preservar os nomes das características



# Previsão de probabilidades para a instância
# o predict_proba calcula a probabilidade de uma instância pertencer a uma das classes possíveis usando um modelo de classificação treinado.
probs = modelo.predict_proba(instancia_test)[0]  # Obtém as probabilidades para cada classe

# O valor de gamma_A é a probabilidade da classe verdadeira
classe_verdadeira = y_test.iloc[0]  # Obtém a classe verdadeira da instância
gamma_A = probs[classe_verdadeira]  # Extrai a probabilidade correspondente à classe verdadeira
print(classe_verdadeira)
print(gamma_A)
# Cálculo do valor delta para cada feature
delta = []
w = modelo.coef_[0]  # pesos do modelo treinado

# Calcula o delta para cada feature
for i, feature in enumerate(df.columns[:-1]):
    if w[i] < 0:
        delta.append((Vs[feature] - df[feature].max()) * w[i])
    else:
        delta.append((Vs[feature] - df[feature].min()) * w[i])

# Calcula R como a soma dos deltas menos gamma_A
R = sum(delta) - gamma_A  # Atualiza R para incluir gamma_A

def one_explanation(Vs, delta, R):
    Xpl = []  # Inicializa a lista de PI-explicação
    # Ordena o delta junto com seus índices, em ordem decrescente de valor absoluto
    delta_sorted = sorted(enumerate(delta), key=lambda x: abs(x[1]), reverse=True)  # enumerando os valores de delta em tuplas e ordenando
    R_atual = R  # Inicializa o limiar atual
    Idx = 0  # Inicializa o índice para iterar
    
    while R_atual >= 0 and Idx < len(delta_sorted):
        sorted_idx, delta_value = delta_sorted[Idx]  # Desempacota o índice e o valor do delta
        feature = X_test.columns[sorted_idx]  # Obtém o nome da feature correspondente
        feature_value = Vs[feature]  # Obtém o valor da feature na instância Vs
        
        # Adiciona à explicação
        Xpl.append(f"{feature} - {feature_value}")  # Adiciona feature e valor à explicação
        
        R_atual -= delta_value  # Atualiza o limiar atual para manter ou parar o loop
        Idx += 1  # Incrementa o índice
    
    return Xpl  # Retorna a PI-explicação

# Computa a PI-explicação
Xpl = one_explanation(Vs, delta, R)

# Imprime os resultados
print(f"Probabilidades: {probs}")
print(f"Classe verdadeira: {classe_verdadeira}")
print(f"Valor de gamma_A: {gamma_A}")
print(f"PI-Explicação: ")
for item in Xpl:
    print(f"- {item}")


1
0.8277146135745481
Probabilidades: [0.00380009 0.82771461 0.16848529]
Classe verdadeira: 1
Valor de gamma_A: 0.8277146135745481
PI-Explicação: 
- petal length (cm) - 4.7
- petal width (cm) - 1.2
- sepal width (cm) - 2.8


In [28]:
teste = -3
soma = abs(teste)
soma

3

In [9]:
# análise completa com todas as instancias do dataset
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carrega o dataset Iris
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)  # organizando em um DataFrame
df['target'] = iris.target  # rótulos

# Divide os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=42)

# Treina o modelo
modelo = LogisticRegression(max_iter=200)
modelo.fit(X_train, y_train)  # Treinando o modelo

# Exibe as classes identificadas pelo modelo
classes = modelo.classes_
print(f"Classes identificadas pelo modelo: {classes}")
print(f"Nomes das classes: {iris['target_names']}")

# Informações sobre o dataset
num_instancias = len(X_test)
print(f"\nNúmero total de instâncias no conjunto de teste: {num_instancias}")

# Escolher qual instância analisar ou todas
# Escolha da instância específica (por exemplo, 0) ou `None` para todas
instancia_para_analisar = 2  
# Operador ternàrio - se a instancia for (None) salva todas na variavel, se não adiciona somente a instância escolhida
instancias_para_analisar = range(num_instancias) if instancia_para_analisar is None else [instancia_para_analisar]
print(instancia_para_analisar)
# Loop para analisar instâncias
for idx in instancias_para_analisar:
    Vs = X_test.iloc[idx].to_dict()  # Utilizando a instância de teste especificada
    instancia_test = X_test.iloc[[idx]]  # Mantém como DataFrame para preservar os nomes das características

    # Previsão de probabilidades para a instância
    probs = modelo.predict_proba(instancia_test)[0]  # Obtém as probabilidades para cada classe usando o 0 para acesar a 1ª linha

    # O valor de gamma_A é a probabilidade da (classe verdadeira - é o rotulo original do dataset o qual pertence a instância)
    classe_verdadeira = y_test.iloc[idx]  # Obtém a classe verdadeira da instância
    gamma_A = probs[classe_verdadeira]  # Extrai a probabilidade correspondente à classe verdadeira treinada pela reg.logistica na func predict_proba

    # Exibe a classe verdadeira e as probabilidades
    print(f"\nInstância {idx}:")
    print(f"Classe verdadeira: {classe_verdadeira} ({iris['target_names'][classe_verdadeira]})")
    print(f"Probabilidades: {probs}")
    print(f"Valor de gamma_A: {gamma_A}")

    # Cálculo dos deltas para cada feature
    delta = []
    w = modelo.coef_[0]  # Pesos do modelo treinado

    for i, feature in enumerate(X_train.columns):
        if w[i] < 0:
            delta.append((Vs[feature] - X_train[feature].max()) * w[i])
        else:
            delta.append((Vs[feature] - X_train[feature].min()) * w[i])

        print('-'*20)
        print(idx)
        print(Vs[feature])
        print(df[feature].max())
        print(df[feature].min())
        print('-'*20)

    # Calcula R como a soma dos deltas menos gamma_A   #### no artigo não menciona se o R pode ou não ser negativo, R negativo sugere que as características                                         não estão explicando suficientemente a predição
    
    R = sum(delta) - gamma_A
    #R = abs(sum(delta) - gamma_A) # a função abs alem de tratar o valor absoluto permite sempre o resultado ser positivo ou seja ela remove o simbolo negativo, porem acredito que está calculando errado

    #R = max(0, sum(delta) - gamma_A)

   
    # Computa a PI-explicação
    Xpl = []
    delta_sorted = sorted(enumerate(delta), key=lambda x: abs(x[1]), reverse=True)
    R_atual = R
    Idx = 0

    # Calcula a explicação
    while R_atual >= 0 and Idx < len(delta_sorted):
        sorted_idx, delta_value = delta_sorted[Idx]
        feature = X_test.columns[sorted_idx]  # Nome da feature correspondente
        feature_value = Vs[feature]  # Valor da feature para a instância

        # Adiciona à explicação
        Xpl.append(f"{feature} - {feature_value}")

        R_atual -= delta_value
        Idx += 1

    # Imprime a PI-explicação
    print(f"Valor de R: {R}")
    print("PI-Explicação:")
    for item in Xpl:
        print(f"- {item}")


Classes identificadas pelo modelo: [0 1 2]
Nomes das classes: ['setosa' 'versicolor' 'virginica']

Número total de instâncias no conjunto de teste: 30
2

Instância 2:
Classe verdadeira: 2 (virginica)
Probabilidades: [8.84412614e-09 1.54875125e-03 9.98451240e-01]
Valor de gamma_A: 0.9984512399032031
--------------------
2
7.7
7.9
4.3
--------------------
--------------------
2
2.6
4.4
2.0
--------------------
--------------------
2
6.9
6.9
1.0
--------------------
--------------------
2
2.3
2.5
0.1
--------------------
Valor de R: -0.69642094092863
PI-Explicação:


In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

iris = load_iris()
df_iris = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df_iris['target'] = iris.target

def pi_explanation(dataset, target_col, instance_index=None):
    # Divide os dados em características (X) e rótulos (y)
    X = dataset.drop(target_col, axis=1)
    y = dataset[target_col]

    # Divide os dados em conjunto de treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Treina o modelo de regressão logística
    modelo = LogisticRegression(max_iter=200)
    modelo.fit(X_train, y_train)  # Treinando o modelo

    # Exibe as classes possíveis
    classes = modelo.classes_  # Identifica as classes
    print(f"Classes identificadas pelo modelo: {classes}")

    # Define quais instâncias serão analisadas (todas ou uma específica)
    if instance_index is not None:
        instances_to_analyze = [instance_index]  # Analisar apenas a instância especificada
    else:
        instances_to_analyze = range(len(X_test))  # Analisar todas as instâncias

    # Loop para analisar uma ou mais instâncias
    for idx in instances_to_analyze:
        Vs = X_test.iloc[idx].to_dict()  # Utilizando a instância de teste especificada
        instancia_test = X_test.iloc[[idx]]  # Mantém como DataFrame para preservar os nomes das características

        # Previsão de probabilidades para a instância
        probs = modelo.predict_proba(instancia_test)[0]  # Obtém as probabilidades para cada classe

        # O valor de gamma_A é a probabilidade da classe verdadeira
        classe_verdadeira = y_test.iloc[idx]  # Obtém a classe verdadeira da instância
        gamma_A = probs[classe_verdadeira]  # Extrai a probabilidade correspondente à classe verdadeira

        # Exibe a classe verdadeira e as probabilidades
        print(f"\nInstância {idx}:")
        print(f"Classe verdadeira: {classe_verdadeira} (Classe {classes[classe_verdadeira]})")
        print(f"Probabilidades: {probs}")
        print(f"Valor de gamma_A: {gamma_A}")

        # Cálculo dos deltas para cada feature
        delta = []
        w = modelo.coef_[0]  # Pesos do modelo treinado

        for i, feature in enumerate(X.columns):
            if w[i] < 0:
                delta.append((Vs[feature] - X[feature].max()) * w[i])
            else:
                delta.append((Vs[feature] - X[feature].min()) * w[i])

        # Calcula R como a soma dos deltas menos gamma_A
        R = sum(delta) - gamma_A

        # Função para computar a PI-explicação
        def one_explanation(Vs, delta, R):
            Xpl = []  # Inicializa a lista de PI-explicação
            delta_sorted = sorted(enumerate(delta), key=lambda x: abs(x[1]), reverse=True)  # Ordena os deltas
            R_atual = R  # Inicializa o limiar atual
            Idx = 0  # Inicializa o índice para iterar

            while R_atual >= 0 and Idx < len(delta_sorted):
                sorted_idx, delta_value = delta_sorted[Idx]  # Desempacota o índice e o valor do delta
                feature = X_test.columns[sorted_idx]  # Obtém o nome da feature correspondente
                feature_value = Vs[feature]  # Obtém o valor da feature na instância Vs

                # Adiciona à explicação
                Xpl.append(f"{feature} - {feature_value}")

                R_atual -= delta_value  # Atualiza o limiar atual
                Idx += 1  # Incrementa o índice
            
            return Xpl  # Retorna a PI-explicação

        # Computa a PI-explicação para a instância
        Xpl = one_explanation(Vs, delta, R)

        # Imprime a PI-explicação
        print("PI-Explicação:")
        for item in Xpl:
            print(f"- {item}")



# Chamada da função para analisar todas as instâncias
pi_explanation(df_iris, target_col='target', instance_index=None)  # Para todas as instâncias

# Chamada da função para analisar uma instância específica
# pi_explanation(df_iris, target_col='target', instance_index=0)  # Para a instância 0


Classes identificadas pelo modelo: [0 1 2]

Instância 0:
Classe verdadeira: 1 (Classe 1)
Probabilidades: [0.00380009 0.82771461 0.16848529]
Valor de gamma_A: 0.8277146135745481
PI-Explicação:
- petal length (cm) - 4.7
- petal width (cm) - 1.2
- sepal width (cm) - 2.8

Instância 1:
Classe verdadeira: 0 (Classe 0)
Probabilidades: [9.46946389e-01 5.30534119e-02 1.99072566e-07]
Valor de gamma_A: 0.9469463890124217
PI-Explicação:
- petal length (cm) - 1.7
- petal width (cm) - 0.3
- sepal width (cm) - 3.8

Instância 2:
Classe verdadeira: 2 (Classe 2)
Probabilidades: [8.84412614e-09 1.54875125e-03 9.98451240e-01]
Valor de gamma_A: 0.9984512399032031
PI-Explicação:

Instância 3:
Classe verdadeira: 1 (Classe 1)
Probabilidades: [0.00647972 0.79218894 0.20133134]
Valor de gamma_A: 0.7921889445681916
PI-Explicação:
- petal length (cm) - 4.5
- petal width (cm) - 1.5
- sepal width (cm) - 2.9

Instância 4:
Classe verdadeira: 1 (Classe 1)
Probabilidades: [0.0014558  0.77408417 0.22446003]
Valor de gam